In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

### 1. Recuperar base já pré-processada

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\\Finais\\enem_microdados_2023.pkl')

In [ ]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']
grupo_previsao = ['NUM_NOTA_CH']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[grupo_previsao]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

In [4]:
# Ajustar as dimensões dos arrays
y_test = y_test.squeeze()
y_train_final = y_train_final.squeeze()
y_eval = y_eval.squeeze()

### 2. Modelo base

In [5]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train_final, 
                y_train_final, 
                eval_set=[(X_eval, y_eval)], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4296
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 40
[LightGBM] [Info] Start training from score 527.881246


In [6]:
# Previsões
y_pred_base = modelo_lgbm.predict(X_test)

In [7]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    parametros={**modelo_lgbm.get_params(), "amostra": X_train_final.shape[0], "tempo": tempo_treino},
                    X_train=X_train_final,
                    y_train=y_train_final,
                    y_test=y_test,
                    y_pred=y_pred_base,
                    variavel_alvo='NUM_NOTA_CH',
                    modelo=modelo_lgbm,
                    nome_modelo='modelo_lgbm_base',
                    descricao_modelo='Modelo LGBMRegressor base')

🏃 View run orderly-squid-275 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/103d90b76ba64bdc8057aa6044df760d
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Erro ao registrar o modelo no MLflow: 'NUM_NOTA_CH'
Rastreamento do MLflow finalizado.


In [8]:
# Avaliação grupo treino
avaliar_modelo(y_train_final, modelo_lgbm.predict(X_train_final), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test, y_pred_base, "teste")

MAE (treino): 53.5294
RMSE (treino): 67.8961
R2 (treino): 0.3564
MAE (teste): 55.2285
RMSE (teste): 70.0224
R2 (teste): 0.3122


In [9]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_lgbm, 'Modelos\\modelo_lgbm_base.pkl')

['Modelos\\modelo_lgbm_base.pkl']

### 3. Bayes Search

In [10]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [11]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (52, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (60, 100),                         # Profundidade máxima da árvore
    'learning_rate': (0.001, 0.01, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                   # Número de árvores
    'subsample': (0.5, 0.9),                        # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.3, 0.9),                 # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1e-1, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-6, 1e-4, 'log-uniform'),      # Regularização L2
}

In [12]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=20,                      # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [13]:
fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],               # Métricas a serem avaliadas
    'categorical_feature': categorical_features,        # Colunas categóricas
}

In [14]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train_final, y_train_final, **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [15]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {
        "colsample_bytree": 0.2297823151086026,
        "learning_rate": 0.028382425255806625,
        "max_depth": 40,
        "n_estimators": 5990,
        "num_leaves": 40,
        "reg_alpha": 0.00596084139054512,
        "reg_lambda": 0.8601375912019539,
        "subsample": 0.9434176158805601
    }
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.30911765737060976), ('learning_rate', 0.009989721326532239), ('max_depth', 96), ('n_estimators', 5166), ('num_leaves', 54), ('reg_alpha', 0.04022442508037429), ('reg_lambda', 6.983061213998744e-05), ('subsample', 0.769767140418842)])
R2:  0.3170481368467291
Tempo total de execução: 25934.18 segundos


In [16]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_final,
                      y_train_final,
                      eval_set=[(X_eval, y_eval)], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4296
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 40
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[4690]	valid_0's rmse: 69.8631	valid_0's l1: 55.0932	valid_0's l2: 4880.86


In [17]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [18]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train_final.shape[0], "tempo": tempo_treino},
                    X_train=X_train_final,
                    y_train=y_train_final,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV')

🏃 View run flawless-deer-773 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/dae7b788b08842a6b8a9965277dd1b45
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Erro ao registrar o modelo no MLflow: 'NUM_NOTA_CH'
Rastreamento do MLflow finalizado.


In [19]:
# Avaliação grupo treino
avaliar_modelo(y_train_final, modelo_lgbm_bayes.predict(X_train_final), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test, y_pred_bayes, "teste")

MAE (treino): 52.7391
RMSE (treino): 66.9660
R2 (treino): 0.3739
MAE (teste): 54.8517
RMSE (teste): 69.6036
R2 (teste): 0.3204


In [20]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_lgbm_bayes, 'Modelos\\modelo_lgbm_bayes.pkl')

['Modelos\\modelo_lgbm_bayes.pkl']